# imports

In [ ]:
import os
import os.path
import pickle
import matplotlib.pyplot as plt

from test_runner import TestCase, BaseTestRunner, LiftedPlanningRunner, GroundedPlanningRunner
from test_runner.tapaal_caller import QueryResult


# Load Data

In [ ]:
results_dir = "./results"
plot_save_dir = "./results/plots"

os.makedirs(plot_save_dir, exist_ok=True)


results: dict[TestCase, dict[BaseTestRunner, list[QueryResult]]] = {}

for filename in os.listdir(results_dir):
    print(f"Loading {filename}")
    full_path = os.path.join(results_dir, filename)
    if os.path.isfile(full_path):
        with open(full_path, "rb") as f:
            data: dict[TestCase, dict[BaseTestRunner, list[QueryResult]]] = pickle.load(f)
            for k,v in data.items():
                results[k] = v

results

# Plots

## General Setup

## Cactus Plot

In [ ]:
# Total Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        # times = [res.time_total for res in query_results]
        times = [res.time_total for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Total Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

plt.savefig(os.path.join(plot_save_dir, "blocksworld_total_time.png"))

In [ ]:
# Tapaal Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.time_tapaal for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Tapaal Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

plt.savefig(os.path.join(plot_save_dir, "blocksworld_tapaal_time.png"))

In [ ]:
# Verification Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.output["time_verification"] for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Verification Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

# plt.show()
plt.savefig(os.path.join(plot_save_dir, "blocksworld_verification_time.png"))

In [ ]:
# Translation Time

fig_rows = 2
fig_cols = 3

i=1
plt.figure(figsize=(10, 6))
for test_case, runners in results.items():
    plt.subplot(fig_rows, fig_cols, i)
    for runner, query_results in runners.items():
        times = [res.time_translation for res in query_results]
        times.sort()
        median = times[int(len(times)/2)]
        plt.plot(times, 'o-', label=f"{runner.translation_name}")  # - {runner.description} - median={median:.4f}")

    plt.title(f'Translation Time - {test_case.name}')
    plt.gca().legend(loc='best')
    plt.xlabel('Index')
    plt.ylabel('Time (sec)')

    #Saving the plots
    # filename = f'plots/{runner}_{test_case}.png'
    # plt.savefig(filename)
    i += 1

# plt.show()
plt.savefig(os.path.join(plot_save_dir, "blocksworld_translation_time.png"))